In [334]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt

In [192]:
print("Tensorflow version: {}".format(tf.__version__))
print("Is GPU available? {}".format(tf.config.list_physical_devices('GPU')))

Tensorflow version: 2.3.1
Is GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Encoding Layer Class

This function is to perform the rnn_conv operation that makes up the building block of the encoder and decoder

In [193]:
B_conv_channels = 32

D_conv1_channels = 512
D_rnn1_channels = 512
D_rnn2_channels = 512
D_rnn3_channels = 256
D_rnn4_channels = 128
D_conv2_channels = 3

In [194]:
def padding(x, stride):
    if x % stride == 0:
        return x // stride
    else:
        return x // stride + 1

In [195]:
data = np.random.normal(size=(128,32,32,3))
ENCODER_DIM = 32

In [196]:
def init_hiddens(data):

    height = padding(padding(data.shape[1], 2),2)
    width = padding(padding(data.shape[2], 2),2)
        
    # height = data.shape[1]
    # width = data.shape[2]
    
    print("height: {}".format(height))
    print("width: {}".format(width))
    
    shape = [data.shape[0]] + [height, width] + [64]
    hidden = tf.zeros(shape)
    cell = tf.zeros(shape)
        
    return (hidden, cell)

In [197]:
hiddens = init_hiddens(data)

print("Hidden shape: {}".format(hiddens[0].shape))
print("Cell shape: {}".format(hiddens[1].shape))

input_data = Conv2D(filters=32, kernel_size=[3,3], strides=(2,2), padding="same")(data)
rnn_conv_ex = RnnConv(input_data, hiddens=hiddens, filters=64,kernel_size=3, strides=2)

height: 8
width: 8
Hidden shape: (128, 8, 8, 64)
Cell shape: (128, 8, 8, 64)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



TypeError: __init__() got an unexpected keyword argument 'hiddens'

In [198]:
conv_inputs = np.load("TF1_data/conv_inputs_epoch5_step60.npy")
conv_hidden = np.load("TF1_data/conv_hidden_epoch5_step60.npy")
cell_state = np.load("TF1_data/hiddens1_epoch5_step60.npy")[1]

In [199]:
in_gate, f_gate, out_gate, c_gate = tf.split(conv_hidden + conv_inputs, 4, axis=-1)
new_cell = tf.multiply(f_gate, cell_state) + tf.multiply(in_gate, c_gate)

### Encoder Class Experimentation

In [200]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [342]:
class RnnConv(Layer):
    
    def __init__(self, filters, kernel_size, strides, index, batch_size):
        
        super(RnnConv, self).__init__()
        self.batch_size = batch_size
        
        self.index = index        
        self.hidden, self.cell = self.init_hiddens(filters)
        
        self.conv_inputs_layers = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding="same")
        self.conv_hidden_layers = Conv2D(filters=filters, kernel_size=kernel_size, padding="same")
        
        self.in_gate = Dense(filters, activation="sigmoid")
        self.f_gate = Dense(filters, activation="sigmoid")
        self.out_gate = Dense(filters, activation="sigmoid")
        self.c_gate = Dense(filters, activation="tanh")
        
        self.hidden_weight = Dense(1, activation="tanh")

    def init_hiddens(self, num_filters):

        h_w_scale_factor = 2**(self.index+1)
        h_w = int(32 // h_w_scale_factor) #32 is the input dims, decreasing by factor of 2 for each layer

        # height = self.padding(data.shape[1], 2) // 2
        # width = self.padding(data.shape[2], 2) // 2

        shape = [self.batch_size] + [h_w, h_w] + [num_filters]
        hidden = tf.zeros(shape)
        cell = tf.zeros(shape)

        return (hidden, cell)
    
    def call(self, inputs):
        
        self.batch_size = inputs.shape[0]
        
        print("inputs size: {}".format(inputs.shape))
        print("hidden size: {}".format(self.hidden.shape))
        
        self.conv_inputs = self.conv_inputs_layers(inputs)
        self.conv_hidden = self.conv_hidden_layers(self.hidden)
        
        print("input after conv: {}".format(self.conv_inputs.shape))
        print("conv_hidden after conv: {}".format(self.conv_hidden.shape))
        print("\n")
        
        in_gate, f_gate, out_gate, c_gate = tf.split(self.conv_inputs + self.conv_hidden, 4, axis=-1)
 
        self.in_gate = self.in_gate(in_gate)
        self.f_gate = self.f_gate(f_gate)
        self.out_gate = self.out_gate(out_gate)
        self.c_gate = self.c_gate(c_gate)
        
        x = tf.multiply(self.f_gate, self.cell)
        
        
        self.new_cell = tf.multiply(self.f_gate, self.cell) + tf.multiply(self.in_gate, self.c_gate)
        self.new_hidden = tf.multiply(self.out_gate, self.hidden_weight(self.new_cell))
        
        self.cell = self.new_cell # should this update the state of the LSTM cell in this layer (and not propagate forward)
        self.hidden = self.new_hidden
        
        return (self.hidden, self.cell)

In [343]:
class Encoder(Layer):
    
    def __init__(self, batch_size):
        super(Encoder, self).__init__()
        
        """
        write defintions of stuff here
        """
        
        self.batch_size = batch_size
        
        # self.batch_size, self.height, self.width, self.channels = images.shape
        self.E_conv_channels = 64
        self.E_rnn1_channels = 256
        self.E_rnn2_channels = 512
        self.E_rnn3_channels = 512
        
        self.input_conv = Conv2D(input_shape=(32,32,3), filters=self.E_conv_channels, kernel_size=3, activation = "relu", strides = (2,2), padding='same')
        self.hiddens1 = RnnConv(filters=self.E_rnn1_channels, kernel_size=3, strides=2, index=1, batch_size=self.batch_size)
        self.hiddens2 = RnnConv(filters=self.E_rnn2_channels, kernel_size=3, strides=2, index=2, batch_size=self.batch_size)
        self.hiddens3 = RnnConv(filters=self.E_rnn3_channels, kernel_size=3, strides=2, index=3, batch_size=self.batch_size)
    
    def padding(self, x, stride):
        
        if x % stride == 0:
            return x // stride
        else:
            return x // stride + 1    
                       
    def call(self, inputs):
        
        self.batch_size = inputs.shape[0]
        
        self.input_conv = self.input_conv(inputs)
                
        self.hiddens1 = self.hiddens1(self.input_conv)        
        self.hiddens2 = self.hiddens2(self.hiddens1[0])
        self.hiddens3 = self.hiddens3(self.hiddens2[0])
                
        return self.hiddens3[0]

In [344]:
encoder_block = Encoder(64)

In [345]:
out = encoder_block(train_images[0:64,:,:,:])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



TypeError: build() takes 0 positional arguments but 2 were given

In [341]:
data = train_images[0:64,:,:,:]

model = Sequential()
model.add(InputLayer(input_shape=(32,32,3)))
model.add(encoder_block)
model.compile(optimizer='adam', loss='mse')

TypeError: in user code:

    <ipython-input-289-eff2a309aeb3>:34 call  *
        self.input_conv = self.input_conv(inputs)

    TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable


In [305]:
var = tf.zeros((64,32,32,3))
var.shape

TensorShape([64, 32, 32, 3])

In [329]:
data = np.random.random(size=(64,32,32,3))